In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/utc_working/medes/')

import os
os.chdir('/content/drive/My Drive/utc_working/medes/')

Mounted at /content/drive


In [2]:
!ls
!pip install -r requirements.txt

250_final_ml100k_user_context_criteria.csv  mha_model.py
427_final_ml1m_user_context_criteria.csv    ml-1m
427_ml1m_statistics_summary.csv		    ml1m_statistics_summary.csv
575_final_ml1m_user_context_criteria.csv    ml_processing.py
575_grouped_ml1m_context_criteria.csv	    mvl_110_plot.pdf
575_ml1m_statistics_summary.csv		    mvl_plot.pdf
average_time.pdf			    outputs
final_ml100k_user_context_criteria.csv	    __pycache__
final_ml1m_user_context_criteria.csv	    requirements.txt
grouped_ml1m_context_criteria.csv	    run.ipynb
imha_model.py				    run_movielen_100k.py
imt_110_plot.pdf			    run_mvdb.py
imt_plot.pdf				    run_mv.sh
ITM-Rec					    run.sh
main.py					    visualization.py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 

In [3]:
!pip install torchinfo
!lscpu
!cat /proc/meminfo | grep Mem
!nvidia-smi --query-gpu=name --format=csv,noheader

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

# **Model Summary**

In [4]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import torch
from torchinfo import summary
import torch.nn as nn
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models.basemodel import BaseModel
from deepctr_torch.layers import PredictionLayer
from deepctr_torch.models import (
    AFM,
    AutoInt,
    CCPM,
    DCN,
    DeepFM,
    DIFM,
    FiBiNET,
    NFM,
    ONN,
    PNN,
    WDL,
    xDeepFM,
    IFM,
    MLR
)
from mha_model import *


# ----------------------------
# Step 1: Load and Merge Data
# ----------------------------
def run(dense_features, sparse_features):
  # Load Data
  df_ratings = pd.read_csv("./250_final_ml100k_user_context_criteria.csv")

  print("df_rating: ", df_ratings.head())

  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])
  df_ratings = df_ratings.dropna()
  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])

  ## Ensure there is at least one argument (excluding script name)

  #if len(sys.argv) < 3:
  #    print("Usage: python main.py <case_number> <log_file>")
  #    sys.exit(1)  # Exit if no argument is provided

  ## Get the argument value and convert it to an integer
  #try:
  #    case = int(sys.argv[1])  # Convert the first argument to an integer
  #    log_file = sys.argv[2]
  #except ValueError:
  #    print("Error: case_number must be an integer.")
  #    sys.exit(1)
  # Redirect stdout and stderr to the log file
  #sys.stdout = open("./outputs/mv_case_1", 'a')
  #sys.stderr = sys.stdout  # Optional: capture errors too


  # ----------------------------
  # Step 2: Feature Engineering
  # ----------------------------

  # Initialize LabelEncoders
  label_encoders = {}

  # Initialize and fit LabelEncoders
  print("sparse_features", sparse_features)
  for feat in sparse_features:
      print("feat: ", feat)
      le = LabelEncoder()
      df_ratings[feat] = le.fit_transform(df_ratings[feat].astype(str))
      label_encoders[feat] = le


  # Initialize MinMaxScaler
  scaler = MinMaxScaler()

  # Fit and transform the dense features
  df_ratings[dense_features] = scaler.fit_transform(df_ratings[dense_features])

  # Define target variables
  #target = ['GroupRating', 'GroupApp', 'GroupData', 'GroupEase']
  target = ['GroupRating']
  # ----------------------------
  # Step 3: Prepare Data for Modeling
  # ----------------------------

  # Split the data
  train_data, test_data = train_test_split(df_ratings, test_size=0.2, random_state=42)

  # Define feature columns
  # Define feature columns
  sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df_ratings[feat].nunique(), embedding_dim=4) for feat in sparse_features]
  dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

  feature_columns = sparse_feature_columns + dense_feature_columns

  print("feature columns: ", feature_columns)
  # Get feature names
  feature_names = get_feature_names(feature_columns)

  # Generate input data for model
  train_model_input = {name: train_data[name].values for name in feature_names}
  test_model_input = {name: test_data[name].values for name in feature_names}

  # Ensure data types are correct
  for name in feature_names:
      if name in sparse_features:
          train_model_input[name] = train_model_input[name].astype(int)
          test_model_input[name] = test_model_input[name].astype(int)
      else:
          train_model_input[name] = train_model_input[name].astype(float)
          test_model_input[name] = test_model_input[name].astype(float)



  # Update the models dictionary
  linear_feature_columns = feature_columns
  dnn_feature_columns =  feature_columns
  num_heads = 4
  dnn_hidden_units = (256, 128, 64, 32, 16, 8)
  l2_reg_dnn = 1e-5
  dnn_dropout = 0.42

  model = MultiHeadAttentionModel(
      linear_feature_columns=linear_feature_columns,
      dnn_feature_columns=dnn_feature_columns,
      num_heads=num_heads,
      dnn_hidden_units=dnn_hidden_units,
      dnn_dropout=dnn_dropout,
      l2_reg_dnn=l2_reg_dnn,
      device="cuda" if torch.cuda.is_available() else "cpu"
  )

  print(model)
  print(summary(model))
sparse_features = [
          'UserID',            # unique user
          'GroupID',           # assigned via KMeans
          'MovieID',           # unique movie
          'Title',             # optional (high-cardinality), usually omit or hash
          'UserOccupation',    # categorical occupation
          'TimeOfDay',         # ['Morning', 'Afternoon', etc.]
          'DayType',           # ['Weekday', 'Weekend']
          'Season'             # ['Spring', 'Summer', etc.]
      ]
dense_features = [
          'UserAge',           # continuous age
          'Storyline',         # simulated multi-criteria
          'Visuals',           # simulated multi-criteria
          'Emotion'           # simulated multi-criteria
      ]
run(dense_features, sparse_features)





df_rating:     UserID  GroupID  MovieID                    Title                  Genres  \
0     196       21      242             Kolya (1996)                  Comedy   
1     196       21      393    Mrs. Doubtfire (1993)                  Comedy   
2     196       21      381  Muriel's Wedding (1994)          Comedy|Romance   
3     196       21      251   Shall We Dance? (1996)                  Comedy   
4     196       21      655       Stand by Me (1986)  Adventure|Comedy|Drama   

   UserAge  UserOccupation  TimeOfDay  DayType  Season  Storyline   Visuals  \
0       49              20  Afternoon  Weekday  Winter   4.986857  5.000000   
1       49              20  Afternoon  Weekday  Winter   3.446943  1.000000   
2       49              20  Afternoon  Weekday  Winter   5.000000  5.000000   
3       49              20  Afternoon  Weekday  Winter   5.000000  1.000000   
4       49              20  Afternoon  Weekday  Winter   4.063386  3.690389   

    Emotion  OverallRating  
0  

# **MCGRS-MC: Running Model Training with all context and criteria**

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import torch
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models.basemodel import BaseModel
from deepctr_torch.layers import PredictionLayer
from deepctr_torch.models import (
    AFM,
    AutoInt,
    CCPM,
    DCN,
    DeepFM,
    DIFM,
    FiBiNET,
    NFM,
    ONN,
    PNN,
    WDL,
    xDeepFM,
    IFM,
    MLR
)
from mha_model import *


# ----------------------------
# Step 1: Load and Merge Data
# ----------------------------
def run(dense_features, sparse_features):
  # Load Data
  df_ratings = pd.read_csv("./250_final_ml100k_user_context_criteria.csv")

  print("df_rating: ", df_ratings.head())

  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])
  df_ratings = df_ratings.dropna()
  print("Rows with NaN values in df_ratings:")
  print(df_ratings[df_ratings.isnull().any(axis=1)])

  ## Ensure there is at least one argument (excluding script name)

  #if len(sys.argv) < 3:
  #    print("Usage: python main.py <case_number> <log_file>")
  #    sys.exit(1)  # Exit if no argument is provided

  ## Get the argument value and convert it to an integer
  #try:
  #    case = int(sys.argv[1])  # Convert the first argument to an integer
  #    log_file = sys.argv[2]
  #except ValueError:
  #    print("Error: case_number must be an integer.")
  #    sys.exit(1)
  # Redirect stdout and stderr to the log file
  #sys.stdout = open("./outputs/mv_case_1", 'a')
  #sys.stderr = sys.stdout  # Optional: capture errors too


  # ----------------------------
  # Step 2: Feature Engineering
  # ----------------------------

  # Initialize LabelEncoders
  label_encoders = {}

  # Initialize and fit LabelEncoders
  print("sparse_features", sparse_features)
  for feat in sparse_features:
      print("feat: ", feat)
      le = LabelEncoder()
      df_ratings[feat] = le.fit_transform(df_ratings[feat].astype(str))
      label_encoders[feat] = le


  # Initialize MinMaxScaler
  scaler = MinMaxScaler()

  # Fit and transform the dense features
  df_ratings[dense_features] = scaler.fit_transform(df_ratings[dense_features])

  # Define target variables
  #target = ['GroupRating', 'GroupApp', 'GroupData', 'GroupEase']
  target = ['OverallRating']
  # ----------------------------
  # Step 3: Prepare Data for Modeling
  # ----------------------------

  # Split the data
  train_data, test_data = train_test_split(df_ratings, test_size=0.2, random_state=42)

  # Define feature columns
  # Define feature columns
  sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df_ratings[feat].nunique(), embedding_dim=4) for feat in sparse_features]
  dense_feature_columns = [DenseFeat(feat, 1) for feat in dense_features]

  feature_columns = sparse_feature_columns + dense_feature_columns

  print("feature columns: ", feature_columns)
  # Get feature names
  feature_names = get_feature_names(feature_columns)

  # Generate input data for model
  train_model_input = {name: train_data[name].values for name in feature_names}
  test_model_input = {name: test_data[name].values for name in feature_names}

  # Ensure data types are correct
  for name in feature_names:
      if name in sparse_features:
          train_model_input[name] = train_model_input[name].astype(int)
          test_model_input[name] = test_model_input[name].astype(int)
      else:
          train_model_input[name] = train_model_input[name].astype(float)
          test_model_input[name] = test_model_input[name].astype(float)

  models = {
            "AFM": (AFM, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,  # AFM still uses DenseFeat here
          'dnn_feature_columns': sparse_feature_columns,  # REMOVE DenseFeat here
          'attention_factor': 8,
          'l2_reg_att': 1e-5,
          'use_attention': True,
      }),
      "AutoInt": (AutoInt, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,
          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,  # Keep both for other models
          'att_layer_num': 3,
          'dnn_hidden_units': (256, 128),
      }),
      "CCPM": (CCPM, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,  # Keep DenseFeat in linear features
          'dnn_feature_columns': sparse_feature_columns,  # REMOVE DenseFeat from dnn_feature_columns
          'conv_kernel_width': (6, 5),
          'conv_filters': (4, 4),
          'dnn_hidden_units': (256, 128),
      }),
      "DCN": (DCN, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,
          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,
          'cross_num': 3,
          'dnn_hidden_units': (256, 128),
      }),
      "DeepFM": (DeepFM, {
          'dnn_feature_columns': feature_columns,
      'linear_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "DIFM": (DIFM, {
          'dnn_feature_columns': feature_columns,
          'linear_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "FiBiNET": (FiBiNET, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "NFM": (NFM, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "ONN": (ONN, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "PNN": (PNN, {
          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,  # Only dnn_feature_columns
          'dnn_hidden_units': (256, 128),
          'use_inner': True,
          'use_outter': False,
      }),
      "WDL": (WDL, {
          'linear_feature_columns': feature_columns,
          'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
      }),
      "xDeepFM": (xDeepFM, {
          'linear_feature_columns': feature_columns,
        'dnn_feature_columns': feature_columns,
          'dnn_hidden_units': (256, 128),
          'cin_layer_size': (128, 128),
      }),
      "IFM": (IFM, {
          'linear_feature_columns': sparse_feature_columns + dense_feature_columns,

          'dnn_feature_columns': sparse_feature_columns + dense_feature_columns,
          'dnn_hidden_units': (256, 128),
      }),

      "LS-PLM": (MLR, {
          'region_feature_columns': sparse_feature_columns + dense_feature_columns,
        'base_feature_columns': sparse_feature_columns,  # Example
        'bias_feature_columns': dense_feature_columns,  # Example
      }),


      "MHA": (MultiHeadAttentionModel, {
        'linear_feature_columns': feature_columns,
        'dnn_feature_columns': feature_columns,
        'num_heads': 4,
        'dnn_hidden_units': (256, 128, 64, 32, 16, 8),
        'l2_reg_dnn': 1e-5,
        'dnn_dropout': 0.42,
      })
  }

  def ndcg_at_k(r, k):
    r = np.asarray(r, dtype=float)[:k]
    if r.size == 0:
      return 0.0
    dcg = np.sum((2 ** r - 1) / np.log2(np.arange(2, r.size + 2)))
    ideal_r = np.sort(r)[::-1]
    idcg = np.sum((2 ** ideal_r - 1) / np.log2(np.arange(2, ideal_r.size + 2)))
    return dcg / idcg if idcg != 0 else 0.0


  def precision_at_k(y_true, y_pred, k, relevance_threshold=3.7):  # Add relevance threshold
      idx = np.argsort(y_pred)[::-1][:k]
      return np.sum(np.array(y_true)[idx] >= relevance_threshold) / k  # Apply threshold

  def recall_at_k(y_true, y_pred, k, relevance_threshold=3.7):  # Add relevance threshold
      relevant = np.sum(np.array(y_true) >= relevance_threshold)  # Apply threshold
      idx = np.argsort(y_pred)[::-1][:k]
      return np.sum(np.array(y_true)[idx] >= relevance_threshold) / relevant if relevant > 0 else np.nan  # Handle zero division

  def f1_at_k(precision, recall):
      return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

  # Initialize results dictionary
  results = {}

  # Device configuration
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # Function to measure CPU & memory usage
  def get_resource_usage():
      process = psutil.Process()
      cpu_percent = psutil.cpu_percent(interval=1)  # CPU usage over 1 second
      memory_info = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
      return cpu_percent, memory_info

  # Train and evaluate each model

  for model_name, (model_class, model_params) in models.items():
      print(f"\nTraining and evaluating {model_name}...")

      # Start measuring time and resource usage
      start_time = time.time()
      cpu_before, memory_before = get_resource_usage()

      # Define common parameters
      common_params = {
          'task': 'regression',
          'device': device
      }
      # Update with model-specific parameters
      common_params.update(model_params)

      # Handle models that require special inputs
      if model_name in ['AFM']:
          common_params['use_attention'] = True

      # Define the model
      model = model_class(**common_params)
      model.to(device)

      # Compile the model
      optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
      model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])

      # Define Early Stopping
      early_stopping = EarlyStopping(monitor='val_mse', patience=5, verbose=1)
      time.sleep(2)  # Simulate some work (replace with model training)
      # Train the model
      history = model.fit(
          train_model_input,
          train_data[target].values,
          batch_size=256,
          epochs=50,
          verbose=2,
          validation_split=0.3,
          callbacks=[early_stopping]
      )

      # Predict on test set
      predictions = model.predict(test_model_input, batch_size=256)

      # Calculate RMSE and MAE
      rmse = np.sqrt(mean_squared_error(test_data[target].values, predictions))
      mae = mean_absolute_error(test_data[target].values, predictions)

      k = 5
      test_df = test_data.copy()
      test_df["prediction"] = predictions

      # Compute NDCG per GroupID
      precisions5, precisions10, recalls, f1s, ndcgs5,ndcgs10 = [], [], [], [], [], []
      for group_id, group_df in test_df.groupby("GroupID"):
          true_ratings = group_df[target[0]].values
          pred_scores = group_df["prediction"].values

          # Sort items by predicted scores
          sorted_idx = np.argsort(pred_scores)[::-1]
          sorted_true = true_ratings[sorted_idx]

          prec5 = precision_at_k(true_ratings, pred_scores, 5)
          prec10 = precision_at_k(true_ratings, pred_scores, 10)


          ndcg5 = ndcg_at_k(sorted_true, 5)
          ndcg10 = ndcg_at_k(sorted_true, 10)


          precisions5.append(prec5)
          precisions10.append(prec10)
          ndcgs5.append(ndcg5)
          ndcgs10.append(ndcg10)

      avg_precision5 = np.mean(precisions5)
      avg_precision10 = np.mean(precisions10)
      avg_ndcg5 = np.mean(ndcgs5)
      avg_ndcg10 = np.mean(ndcgs10)

      # Convert MSE to RMSE and store both RMSE & MAE in history
      history.history['RMSE'] = np.sqrt(history.history['mse'])  # Convert MSE to RMSE
      history.history['val_RMSE'] = np.sqrt(history.history['val_mse'])  # Convert validation MSE to RMSE


      # Measure execution time and resource usage
      end_time = time.time()
      cpu_after, memory_after = get_resource_usage()
      execution_time = end_time - start_time
      cpu_usage = cpu_after - cpu_before
      memory_usage = memory_after - memory_before

      # Store the results
      results[model_name] = {
          'RMSE': rmse,
          'MAE': mae,
          'NDCG@5':avg_ndcg5,
          'NDCG@10':avg_ndcg10,
          'Precision@5': avg_precision5,
          'Precision@10': avg_precision10,
          'Execution Time (s)': execution_time,
          'CPU Usage (%)': cpu_usage,
          'Memory Usage (MB)': memory_usage
      }

      print(f"{model_name} Model RMSE: {rmse:.4f}")
      print(f"{model_name} Model MAE: {mae:.4f}")
      print(f"{model_name} Model NDCG@5: {avg_ndcg5:.4f}")
      print(f"{model_name} Model NDCG@10: {avg_ndcg10:.4f}")
      print(f"{model_name} Model Precision@5: {avg_precision5:.4f}")
      print(f"{model_name} Model Precision@10: {avg_precision10:.4f}")
      print(f"{model_name} Execution Time: {execution_time:.2f} seconds")
      print(f"{model_name} CPU Usage: {cpu_usage:.2f}%")
      print(f"{model_name} Memory Usage: {memory_usage:.2f} MB")

      # Save the history for MAE and RMSE after training each model
      with open(f'./outputs/pickles/{model_name}_mvl_history.pkl', 'wb') as file_pi:
          pickle.dump(history.history, file_pi)

  # Display all results
  print("\nFinal Results:")
  for model_name, metrics in results.items():
      #print(f"{model_name} - RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}, "
      #      f"Time: {metrics['Execution Time (s)']:.2f}s, CPU: {metrics['CPU Usage (%)']:.2f}%, "
      #      f"Memory: {metrics['Memory Usage (MB)']:.2f}MB")
      print(f"{model_name} - RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}, ",
        f"NDCG@5: {metrics['NDCG@5']:.4f}, ", f"Precision@5: {metrics['Precision@5']:.4f},",
            f"NDCG@10: {metrics['NDCG@10']:.4f}, ",
            f"Precision@10: {metrics['Precision@10']:.4f}, ",
            f"Time: {metrics['Execution Time (s)']:.2f}s, ",       f"CPU: {metrics['CPU Usage (%)']:.2f}%, Memory: {metrics['Memory Usage (MB)']:.2f}MB")
  #sys.stdout.close()
  #sys.stdout = sys.__stdout__

  case = 1
  # Decision based on case value
sparse_features = [
          'UserID',            # unique user
          'GroupID',           # assigned via KMeans
          'MovieID',           # unique movie
          'Title',             # optional (high-cardinality), usually omit or hash
          'UserOccupation',    # categorical occupation
          'TimeOfDay',         # ['Morning', 'Afternoon', etc.]
          'DayType',           # ['Weekday', 'Weekend']
          'Season'             # ['Spring', 'Summer', etc.]
      ]
dense_features = [
          'UserAge',           # continuous age
          'Storyline',         # simulated multi-criteria
          'Visuals',           # simulated multi-criteria
          'Emotion'           # simulated multi-criteria
      ]
run(dense_features, sparse_features)

df_rating:     UserID  GroupID  MovieID                    Title                  Genres  \
0     196       21      242             Kolya (1996)                  Comedy   
1     196       21      393    Mrs. Doubtfire (1993)                  Comedy   
2     196       21      381  Muriel's Wedding (1994)          Comedy|Romance   
3     196       21      251   Shall We Dance? (1996)                  Comedy   
4     196       21      655       Stand by Me (1986)  Adventure|Comedy|Drama   

   UserAge  UserOccupation  TimeOfDay  DayType  Season  Storyline   Visuals  \
0       49              20  Afternoon  Weekday  Winter   4.986857  5.000000   
1       49              20  Afternoon  Weekday  Winter   3.446943  1.000000   
2       49              20  Afternoon  Weekday  Winter   5.000000  5.000000   
3       49              20  Afternoon  Weekday  Winter   5.000000  1.000000   
4       49              20  Afternoon  Weekday  Winter   4.063386  3.690389   

    Emotion  OverallRating  
0  

# **MCGRS: Running Prediction with criteria, without context**

In [ ]:
case = 2
  # Decision based on case value
sparse_features = [
          'UserID',            # unique user
          'GroupID',           # assigned via KMeans
          'MovieID',           # unique movie
          'Title',             # optional (high-cardinality), usually omit or hash
          'UserOccupation',    # categorical occupation
      ]
dense_features = [
          'UserAge',           # continuous age
          'Storyline',         # simulated multi-criteria
          'Visuals',           # simulated multi-criteria
          'Emotion'           # simulated multi-criteria
      ]
run(dense_features, sparse_features)

df_rating:     UserID  GroupID  MovieID                    Title                  Genres  \
0     196       21      242             Kolya (1996)                  Comedy   
1     196       21      393    Mrs. Doubtfire (1993)                  Comedy   
2     196       21      381  Muriel's Wedding (1994)          Comedy|Romance   
3     196       21      251   Shall We Dance? (1996)                  Comedy   
4     196       21      655       Stand by Me (1986)  Adventure|Comedy|Drama   

   UserAge  UserOccupation  TimeOfDay  DayType  Season  Storyline   Visuals  \
0       49              20  Afternoon  Weekday  Winter   4.986857  5.000000   
1       49              20  Afternoon  Weekday  Winter   3.446943  1.000000   
2       49              20  Afternoon  Weekday  Winter   5.000000  5.000000   
3       49              20  Afternoon  Weekday  Winter   5.000000  1.000000   
4       49              20  Afternoon  Weekday  Winter   4.063386  3.690389   

    Emotion  OverallRating  
0  

# **GRS: Running Model Training without all context and criteria**

In [ ]:
case = 2
  # Decision based on case value
sparse_features = [
          'UserID',            # unique user
          'GroupID',           # assigned via KMeans
          'MovieID',           # unique movie
          'Title',             # optional (high-cardinality), usually omit or hash
          'UserOccupation',    # categorical occupation
      ]
dense_features = [
          'UserAge',           # continuous age
      ]
run(dense_features, sparse_features)

df_rating:     UserID  GroupID  MovieID                    Title                  Genres  \
0     196       21      242             Kolya (1996)                  Comedy   
1     196       21      393    Mrs. Doubtfire (1993)                  Comedy   
2     196       21      381  Muriel's Wedding (1994)          Comedy|Romance   
3     196       21      251   Shall We Dance? (1996)                  Comedy   
4     196       21      655       Stand by Me (1986)  Adventure|Comedy|Drama   

   UserAge  UserOccupation  TimeOfDay  DayType  Season  Storyline   Visuals  \
0       49              20  Afternoon  Weekday  Winter   4.986857  5.000000   
1       49              20  Afternoon  Weekday  Winter   3.446943  1.000000   
2       49              20  Afternoon  Weekday  Winter   5.000000  5.000000   
3       49              20  Afternoon  Weekday  Winter   5.000000  1.000000   
4       49              20  Afternoon  Weekday  Winter   4.063386  3.690389   

    Emotion  OverallRating  
0  

# **MCGRS-SC-T: Running Prediction with criteria, with single context: TimeOfDay**

In [ ]:
sparse_features = [
          'UserID',            # unique user
          'GroupID',           # assigned via KMeans
          'MovieID',           # unique movie
          'Title',             # optional (high-cardinality), usually omit or hash
          'UserOccupation',    # categorical occupation
          'TimeOfDay'         # ['Morning', 'Afternoon', etc.]
      ]
dense_features = [
          'UserAge',           # continuous age
          'Storyline',         # simulated multi-criteria
          'Visuals',           # simulated multi-criteria
          'Emotion'           # simulated multi-criteria
      ]
run(dense_features, sparse_features)

df_rating:     UserID  GroupID  MovieID                    Title                  Genres  \
0     196       21      242             Kolya (1996)                  Comedy   
1     196       21      393    Mrs. Doubtfire (1993)                  Comedy   
2     196       21      381  Muriel's Wedding (1994)          Comedy|Romance   
3     196       21      251   Shall We Dance? (1996)                  Comedy   
4     196       21      655       Stand by Me (1986)  Adventure|Comedy|Drama   

   UserAge  UserOccupation  TimeOfDay  DayType  Season  Storyline   Visuals  \
0       49              20  Afternoon  Weekday  Winter   4.986857  5.000000   
1       49              20  Afternoon  Weekday  Winter   3.446943  1.000000   
2       49              20  Afternoon  Weekday  Winter   5.000000  5.000000   
3       49              20  Afternoon  Weekday  Winter   5.000000  1.000000   
4       49              20  Afternoon  Weekday  Winter   4.063386  3.690389   

    Emotion  OverallRating  
0  

# **MCGRS-SC-D: Running Prediction with criteria, with single context: DayType**

In [ ]:
sparse_features = [
          'UserID',            # unique user
          'GroupID',           # assigned via KMeans
          'MovieID',           # unique movie
          'Title',             # optional (high-cardinality), usually omit or hash
          'UserOccupation',    # categorical occupation
          'DayType'           # ['Weekday', 'Weekend']
      ]
dense_features = [
          'UserAge',           # continuous age
          'Storyline',         # simulated multi-criteria
          'Visuals',           # simulated multi-criteria
          'Emotion'           # simulated multi-criteria
      ]
run(dense_features, sparse_features)

df_rating:     UserID  GroupID  MovieID                    Title                  Genres  \
0     196       21      242             Kolya (1996)                  Comedy   
1     196       21      393    Mrs. Doubtfire (1993)                  Comedy   
2     196       21      381  Muriel's Wedding (1994)          Comedy|Romance   
3     196       21      251   Shall We Dance? (1996)                  Comedy   
4     196       21      655       Stand by Me (1986)  Adventure|Comedy|Drama   

   UserAge  UserOccupation  TimeOfDay  DayType  Season  Storyline   Visuals  \
0       49              20  Afternoon  Weekday  Winter   4.986857  5.000000   
1       49              20  Afternoon  Weekday  Winter   3.446943  1.000000   
2       49              20  Afternoon  Weekday  Winter   5.000000  5.000000   
3       49              20  Afternoon  Weekday  Winter   5.000000  1.000000   
4       49              20  Afternoon  Weekday  Winter   4.063386  3.690389   

    Emotion  OverallRating  
0  

# **MCGRS-SC-Se: Running Prediction with criteria, with single context: Season**

In [ ]:
sparse_features = [
          'UserID',            # unique user
          'GroupID',           # assigned via KMeans
          'MovieID',           # unique movie
          'Title',             # optional (high-cardinality), usually omit or hash
          'UserOccupation',    # categorical occupation
          'Season'             # ['Spring', 'Summer', etc.]
      ]
dense_features = [
          'UserAge',           # continuous age
          'Storyline',         # simulated multi-criteria
          'Visuals',           # simulated multi-criteria
          'Emotion'           # simulated multi-criteria
      ]
run(dense_features, sparse_features)

df_rating:     UserID  GroupID  MovieID                    Title                  Genres  \
0     196       21      242             Kolya (1996)                  Comedy   
1     196       21      393    Mrs. Doubtfire (1993)                  Comedy   
2     196       21      381  Muriel's Wedding (1994)          Comedy|Romance   
3     196       21      251   Shall We Dance? (1996)                  Comedy   
4     196       21      655       Stand by Me (1986)  Adventure|Comedy|Drama   

   UserAge  UserOccupation  TimeOfDay  DayType  Season  Storyline   Visuals  \
0       49              20  Afternoon  Weekday  Winter   4.986857  5.000000   
1       49              20  Afternoon  Weekday  Winter   3.446943  1.000000   
2       49              20  Afternoon  Weekday  Winter   5.000000  5.000000   
3       49              20  Afternoon  Weekday  Winter   5.000000  1.000000   
4       49              20  Afternoon  Weekday  Winter   4.063386  3.690389   

    Emotion  OverallRating  
0  

In [ ]:
import os
import json
import psutil
import optuna
import pandas as pd
import numpy as np
import torch

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection   import train_test_split
from sklearn.metrics           import mean_squared_error, mean_absolute_error

from deepctr_torch.callbacks import EarlyStopping
from deepctr_torch.inputs    import SparseFeat, DenseFeat, get_feature_names
from mha_model               import MultiHeadAttentionModel

# ----------------------------
# Step 0: Configuration
# ----------------------------
DATA_PATH       = "./427_final_ml1m_user_context_criteria.csv"
TARGET_COLUMN   = "OverallRating"
SPARSE_FEATURES = [
    'UserID','GroupID','MovieID','Title','Genres',
    'UserOccupation','TimeOfDay','DayType','Season'
]
DENSE_FEATURES  = ['UserAge','Storyline','Visuals','Emotion']
DEVICE          = 'cuda' if torch.cuda.is_available() else 'cpu'
N_TRIALS        = 30
RANDOM_SEED     = 42

# Fix random seeds
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# ----------------------------
# Step 1: Load & Preprocess
# ----------------------------
df = pd.read_csv(DATA_PATH).dropna()

# Encode categorical features
for feat in SPARSE_FEATURES:
    encoder = LabelEncoder()
    df[feat] = encoder.fit_transform(df[feat].astype(str))

# Scale continuous features to [0,1]
scaler = MinMaxScaler()
df[DENSE_FEATURES] = scaler.fit_transform(df[DENSE_FEATURES])

# Split into train & test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

# ----------------------------
# Step 2: Build DeepCTR Inputs
# ----------------------------
sparse_columns = [
    SparseFeat(feat, vocabulary_size=df[feat].nunique(), embedding_dim=8)
    for feat in SPARSE_FEATURES
]
dense_columns = [DenseFeat(feat, 1) for feat in DENSE_FEATURES]
feature_columns = sparse_columns + dense_columns
feature_names   = get_feature_names(feature_columns)

def build_input(df_):
    """Convert DataFrame to DeepCTR input dict."""
    model_input = {name: df_[name].values for name in feature_names}
    for name in feature_names:
        if name in SPARSE_FEATURES:
            model_input[name] = model_input[name].astype(int)
        else:
            model_input[name] = model_input[name].astype(float)
    return model_input

train_input = build_input(train_df)
test_input  = build_input(test_df)

# ----------------------------
# Step 3: Ranking Metrics
# ----------------------------
def ndcg_at_k(ratings, k):
    r = np.asanyarray(ratings, dtype=float)[:k]
    if r.size == 0:
        return 0.0
    dcg = np.sum((2**r - 1) / np.log2(np.arange(2, r.size + 2)))
    ideal = np.sort(r)[::-1]
    idcg = np.sum((2**ideal - 1) / np.log2(np.arange(2, ideal.size + 2)))
    return dcg / idcg if idcg > 0 else 0.0

def precision_at_k(y_true, y_pred, k, threshold=3.7):
    idx = np.argsort(y_pred)[::-1][:k]
    return np.sum(np.array(y_true)[idx] >= threshold) / k

# ----------------------------
# Step 4: Optuna Objective
# ----------------------------
def objective(trial):
    # Sample hyperparameters
    num_heads   = trial.suggest_categorical("num_heads",      [2, 4, 8])
    dnn_units   = trial.suggest_categorical("dnn_hidden_units", [
                    (128,64,32), (256,128,64,32), (512,256)
                  ])
    dropout     = trial.suggest_float("dnn_dropout", 0.2, 0.6)
    l2_reg      = trial.suggest_loguniform("l2_reg_dnn", 1e-6, 1e-4)
    lr          = trial.suggest_loguniform("lr",       1e-4, 1e-2)
    batch_size  = trial.suggest_categorical("batch_size", [128, 256, 512])
    patience    = trial.suggest_int("patience", 3, 7)

    # Build model
    model = MultiHeadAttentionModel(
        linear_feature_columns=feature_columns,
        dnn_feature_columns   =feature_columns,
        num_heads             =num_heads,
        dnn_hidden_units      =dnn_units,
        dnn_dropout           =dropout,
        l2_reg_dnn            =l2_reg,
        task                  ='regression',
        device                =DEVICE
    ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])

    # Train with early stopping
    stopper = EarlyStopping(monitor="val_loss", patience=patience, verbose=0)
    history = model.fit(
        train_input, train_df[[TARGET_COLUMN]].values,
        batch_size     =batch_size,
        epochs         =50,
        verbose        =0,
        validation_split=0.2,
        callbacks      =[stopper]
    )

    # Return the final validation MSE
    return history.history["val_mse"][-1]

# ----------------------------
# Step 5: Run Hyperparameter Search
# ----------------------------
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(objective, n_trials=N_TRIALS)

    print(f"\n▶ Best val_mse: {study.best_value:.6f}")
    print("▶ Best hyperparameters:")
    for key, val in study.best_params.items():
        print(f"   • {key} = {val}")

    # Save best parameters
    with open("best_mha_params.json", "w") as f:
        json.dump(study.best_params, f, indent=2)

    # ----------------------------
    # Step 6: Retrain & Final Evaluate
    # ----------------------------
    best = study.best_params
    # Rebuild model with best params
    final_model = MultiHeadAttentionModel(
        linear_feature_columns=feature_columns,
        dnn_feature_columns   =feature_columns,
        num_heads             =best["num_heads"],
        dnn_hidden_units      =tuple(best["dnn_hidden_units"]),
        dnn_dropout           =best["dnn_dropout"],
        l2_reg_dnn            =best["l2_reg_dnn"],
        task                  ='regression',
        device                =DEVICE
    ).to(DEVICE)

    optimizer = torch.optim.Adam(final_model.parameters(), lr=best["lr"])
    final_model.compile(optimizer=optimizer, loss="mse", metrics=["mse","mae"])

    stopper = EarlyStopping(monitor="val_loss", patience=best["patience"], verbose=1)
    final_model.fit(
        train_input, train_df[[TARGET_COLUMN]].values,
        batch_size     =best["batch_size"],
        epochs         =50,
        verbose        =2,
        validation_split=0.1,
        callbacks      =[stopper]
    )

    # Predict on test set
    preds = final_model.predict(test_input, batch_size=best["batch_size"])
    rmse  = np.sqrt(mean_squared_error(test_df[[TARGET_COLUMN]].values, preds))
    mae   = mean_absolute_error(test_df[[TARGET_COLUMN]].values, preds)

    # Compute ranking metrics group-wise
    ndcg5_list, prec5_list = [], []
    test_df["pred"] = preds
    for _, group in test_df.groupby("GroupID"):
        true = group[TARGET_COLUMN].values
        score = group["pred"].values
        ndcg5_list.append(ndcg_at_k(true[np.argsort(score)[::-1]], 5))
        prec5_list.append(precision_at_k(true, score, 5))

    print(f"\n▶ Test RMSE: {rmse:.4f}, MAE: {mae:.4f}")
    print(f"  Test NDCG@5: {np.mean(ndcg5_list):.4f}, Precision@5: {np.mean(prec5_list):.4f}")

    # Save final model weights
    os.makedirs("outputs", exist_ok=True)
    torch.save(final_model.state_dict(), "outputs/best_mha_model.pth")
    print("\n✔ Saved best model to outputs/best_mha_model.pth")


[I 2025-05-09 07:48:05,570] A new study created in memory with name: no-name-046e9b52-d9db-45ff-a12d-4bd72dd739e4
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (256, 128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (512, 256) which is of type tuple.
  warnings.warn(message)
<ipython-input-6-b0b7e50d9418>:102: FutureWarning: suggest_logunifor

cpu
Train on 640133 samples, validate on 160034 samples, 1251 steps per epoch


[I 2025-05-09 08:57:43,192] Trial 0 finished with value: 0.7510999151725549 and parameters: {'num_heads': 4, 'dnn_hidden_units': (128, 64, 32), 'dnn_dropout': 0.2232334448672798, 'l2_reg_dnn': 5.39948440978744e-05, 'lr': 0.0015930522616241021, 'batch_size': 512, 'patience': 7}. Best is trial 0 with value: 0.7510999151725549.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (256, 128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution sho

cpu
Train on 640133 samples, validate on 160034 samples, 1251 steps per epoch


[I 2025-05-09 09:39:47,672] Trial 1 finished with value: 0.7925401473319653 and parameters: {'num_heads': 2, 'dnn_hidden_units': (256, 128, 64, 32), 'dnn_dropout': 0.31649165607921675, 'l2_reg_dnn': 1.6738085788752145e-05, 'lr': 0.00019010245319870352, 'batch_size': 512, 'patience': 6}. Best is trial 0 with value: 0.7510999151725549.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (256, 128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distrib

...

cpu
Train on 640133 samples, validate on 160034 samples, 2501 steps per epoch


[I 2025-05-09 23:01:26,460] Trial 9 finished with value: 0.7731546024243036 and parameters: {'num_heads': 4, 'dnn_hidden_units': (256, 128, 64, 32), 'dnn_dropout': 0.2440207698110707, 'l2_reg_dnn': 2.856737429847189e-06, 'lr': 0.0007148510793512986, 'batch_size': 256, 'patience': 5}. Best is trial 2 with value: 0.7381910644655824.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (256, 128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distributi

cpu
Train on 640133 samples, validate on 160034 samples, 5002 steps per epoch


[I 2025-05-10 06:18:36,960] Trial 10 finished with value: 0.7084596257198429 and parameters: {'num_heads': 8, 'dnn_hidden_units': (512, 256), 'dnn_dropout': 0.5878148443151463, 'l2_reg_dnn': 8.13372824372842e-05, 'lr': 0.009285225829404777, 'batch_size': 128, 'patience': 6}. Best is trial 10 with value: 0.7084596257198429.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (256, 128, 64, 32) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:515: UserWarning: Choices for a categorical distribution shoul

cpu
Train on 640133 samples, validate on 160034 samples, 5002 steps per epoch
